In [1]:
 import pyparsing as pp

def process_template(template):
    # Define pyparsing grammar for commands and content
    command_start = pp.Suppress("{{")
    command_end = pp.Suppress("}}")
    command_name = pp.Word(pp.alphas, pp.alphanums + "_")
    command = command_start + command_name +  command_end
    content = pp.Regex(r"[^{}]+")

    # Parse the template
    parsed_template = pp.OneOrMore(command | content).parseString(template)

    # Process the parsed elements
    user_segment = ""
    assistant_segment = []
    gen_command_count = 0
    for element in parsed_template:
        if isinstance(element, pp.ParseResults):
            if (element["command"]|element["content"]).startswith("gen"):
                if gen_command_count == 2:
                    continue
                gen_command_count += 1
            assistant_segment.append(element)
        else:
            user_segment += element

    assistant_segment = ''.join(assistant_segment)

    if not assistant_segment.endswith("{{gen 'rewrite' }}"):
        assistant_segment += "{{gen 'rewrite' }}"

    assistant_segment = "{{#assistant}}"+"{}".format(assistant_segment)+"{{/assistant}}"

    return ("{{#user}}"+"{}".format(user_segment)+"{{/user}}"+ "{}".format( assistant_segment))

In [2]:
template = "Tweak this proverb to apply to model instructions instead. Where there is no guidance {{gen 'rewrite'}}"

processed_template = process_template(template)

print(processed_template)

{{#user}}Tweak this proverb to apply to model instructions instead. Where there is no guidance {{/user}}{{#assistant}}{{gen 'rewrite' }}{{/assistant}}


In [5]:
template = "how are things going, tell me about Delhi"
processed_template = process_template(template)

print(processed_template)

{{#user}}how are things going, tell me about Delhi{{/user}}{{#assistant}}{{gen 'rewrite' }}{{/assistant}}
